In [22]:
from nipype.interfaces.fsl import epi
from nilearn import plotting
%matplotlib inline
from niwidgets import NiftiWidget
import ipywidgets as widgets
import warnings
warnings.filterwarnings('ignore')
from IPython.display import display
from IPython.display import Javascript
import nilearn.plotting as nip#for using nilearn plot functions
from nilearn.image.image import mean_img

In [3]:
data_path = "../../datos/*"
anat = "mprage.nii.gz"
bold = "f1.nii.gz"
templates_path = "../../datos/TEMPLATES/*"

In [4]:
list_subjs = !ls -d $data_path | grep -v TEMPLATES

In [5]:
subj_id_selector = widgets.Select(
    options=list_subjs,
    # rows=10,
    description='Subject Id:',
    disabled=False
)
#subj_id_selector.on_trait_change(run_cell)
display(subj_id_selector)

Select(description='Subject Id:', options=('../../datos/C021', '../../datos/C024', '../../datos/C026', '../../…

In [15]:
anat_filename = subj_id_selector.value+"/"+anat
bold_filename = subj_id_selector.value+"/"+bold

In [16]:
results_path = "../../results/output/preproc/*"

In [17]:
list_results = !ls -d $results_path | grep session

In [18]:
res_id_selector = widgets.Select(
    options=list_results,
    # value='C575',
    # rows=10,
    description='Subject Id:',
    disabled=False
)
#subj_id_selector.on_trait_change(run_cells_below)
display(res_id_selector)

Select(description='Subject Id:', options=('../../results/output/preproc/_session_id_1_subject_id_C115', '../.…

In [23]:
ref_head = res_id_selector.value+"/Normalize/mprage_ROI_flirt.nii.gz"
ref_brain = res_id_selector.value+"/skull_strip/mprage_ROI_flirt_brain.nii.gz"
mni_3mm_template = "../../datos/TEMPLATES/MNI152_T1_3mm.nii.gz"

In [24]:
(bold_filename,ref_head,ref_brain)

('../../datos/C503/f1.nii.gz',
 '../../results/output/preproc/_session_id_1_subject_id_C503/Normalize/mprage_ROI_flirt.nii.gz',
 '../../results/output/preproc/_session_id_1_subject_id_C503/skull_strip/mprage_ROI_flirt_brain.nii.gz')

In [25]:
fnt = epi.EpiReg()
fnt.inputs.t1_head = ref_head
fnt.inputs.t1_brain = ref_brain
fnt.inputs.epi = bold_filename
fnt.verbose = True
reg_img = fnt.run()

191013-22:36:15,149 nipype.interface INFO:
	 stdout 2019-10-13T22:36:15.140676:Running FAST segmentation
191013-23:35:00,360 nipype.interface INFO:
	 stdout 2019-10-13T23:35:00.356946:FLIRT pre-alignment
191013-23:40:20,670 nipype.interface INFO:
	 stdout 2019-10-13T23:40:20.669061:Running BBR
191013-23:41:16,238 nipype.interface INFO:
	 stderr 2019-10-13T23:41:16.238350:Image Exception : #75 :: 3D only method called by higher-dimensional volume.
191013-23:41:16,375 nipype.interface INFO:
	 stderr 2019-10-13T23:41:16.375087:3D only method called by higher-dimensional volume.
191013-23:41:16,628 nipype.interface INFO:
	 stderr 2019-10-13T23:41:16.628399:Could not open matrix file epi2struct.mat


KeyboardInterrupt: 

In [ ]:
mean_fmri = mean_img(reg_img.outputs.out_file)

In [ ]:
roi_widget = NiftiWidget(mean_fmri)
roi_widget.nifti_plotter(plotting_func=nip.plot_roi, bg_img=mni_3mm_template)